In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from __future__ import division
import matplotlib.pyplot as plt

In [2]:
def SRK(v, P, T, Pc, Tc, w):
    # in K, atm, l/gmol
    R = 0.08206
    s = 0.48 + 1.574*w - 0.176*w**2
    alpha = (1 + s*(1 - (T/Tc)**0.5))**2
    a = 0.42748*R**2*Tc**2*alpha/Pc
    b = 0.08664*R*Tc/Pc
    return v**3 - (R*T/P)*v**2 + ((a-R*T*b)/P - b**2)*v - a*b/P 

In [3]:
def VW(v, P, T, Pc, Tc, w ):
    # in K, atm, l/gmol
    R=0.08206
    a=27*R**2*Tc**2/(64*Pc)
    b=R*Tc/(8*Pc)
    return v**3 - (b + R*T/P)*v**2 + (a/P)*v - a*b/P

In [4]:
def ideal(P, T):
    R = 0.08206
    v_ideal = R*T/P
    return v_ideal

In [5]:
def VSOLVE(P, T, Pc, Tc, EOS, w):
    v_guess = ideal(P, T)
    if EOS == "SRK":
        vsc = fsolve(SRK, v_guess,args = (P, T, Pc, Tc, w))
    elif EOS == "VDW":
        vsc = fsolve(VW, v_guess,args = (P, T, Pc, Tc, w))
    return vsc  

In [6]:
def fugacity(V, P, T, Pc, Tc, EOS, w ):
    R = 0.08206
    if EOS == "VDW":
        a = 27*R**2*Tc**2/(64*Pc)
        b=R*Tc/(8*Pc)
        ln_phi = b/(V-b) - 2*a/(R*T*V) - np.log((P*V)*(1 - b/V)/(R*T))
        phi = np.exp(ln_phi)
    elif EOS == "SRK":
        s = 0.48 + 1.574*w - 0.176*w**2
        alpha = (1 + s*(1 - (T/Tc)**0.5))**2
        a = 0.42748*R**2*Tc**2*alpha/Pc
        b = 0.08664*R*Tc/Pc
        ln_phi = (P*V)/(R*T) -1 - np.log((P*V)/(R*T)*(1 - (b/V))) - ((a*alpha)/(b*R*T)*np.log(1 + (b/V)))
        phi = np.exp(ln_phi)
    return phi    

In [7]:
def fug_maxx(P, T, Pc, Tc, w):
    
    v_srk = VSOLVE(P, T, Pc, Tc, "SRK", w)
    v_vdw = VSOLVE(P, T, Pc, Tc, "VDW", w)
    
    fug_srk = fugacity(v_srk, P, T, Pc, Tc, "SRK", w)
    fug_vdw = fugacity(v_vdw, P, T, Pc, Tc, "VDW", w)
    
    return fug_srk, fug_vdw

In [20]:
co2 = pd.read_csv("co2_f.csv")
water = pd.read_csv("water_f.csv")

co2_numpy = np.array(co2[:])
water_numpy = np.array(water[:])

In [21]:
fug_co2 = []
for i in range(len(co2_numpy)):
    j = fug_maxx(co2_numpy[i][0], co2_numpy[i][1], 72.8053, 304.13, co2_numpy[i][3])
    fug_co2.append(j)

In [44]:
fugco2 = pd.DataFrame(fug_co2)
fugco2.to_csv("f_co2_out.csv")
fugco2

,0,1
0,[0.949748470951674],[0.9572425065725916]
1,[0.9638209150923739],[0.9640771148936281]
2,[0.9873393208289997],[0.9776951615892027]
3,[0.8997836420164126],[0.9142573619915686]
4,[0.927953175264845],[0.9280163266380511]
5,[0.9749101676422163],[0.955358117835517]
6,[0.8016515122888427],[0.8284445206066015]
7,[0.8584095848652805],[0.8565161180973693]
8,[0.9512744785043539],[0.9114330012020009]
9,[0.7546195307012318],[0.74047874448438]


In [48]:
fug_water = []
for i in range(len(water_numpy)):
    j = fug_maxx(water_numpy[i][0], water_numpy[i][1], 217.75474, 647.1, water_numpy[i][3])
    fug_water.append(j)

In [49]:
fug_water = pd.DataFrame(fug_water)
fug_water.to_csv("f_water_out.csv")
fug_water

,0,1
0,[0.9973491256783042],[0.9982025517349682]
1,[0.543056233456384],[0.5145902791038739]
2,[0.8044181440866847],[0.6680467132071279]
3,[0.9284195748084207],[0.7638937446799918]
4,[0.9948402144486442],[0.826178637223781]
5,[1.0305552012852826],[0.8697745560738148]
6,[0.47667903920025123],[0.4543479981864944]
7,[0.7700440651953897],[0.6029623012437649]
8,[0.9180750648898509],[0.7136637354500489]
9,[0.997951469973495],[0.7889104871772601]
